# Laboratorio 7: Redes LSTM

Construiremos dos ejemplos de redes neuronales recurrentes utilizando LSTM.

## 1. Descargar las base de datos de dos series de tiempo

In [ ]:
#https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv
#https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv

## 2. Cargar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

## 3. Cargando y revisando los datos de pasajeros de una aerolínea

Leer los datos utilizando Pandas.

Graficamos los datos.

## 4. Creación de ventana deslizante

Creamos una ventana deslizante, dada una secuencia de tamaño n se desea predecir el siguiente punto.

## 5. Seleccionamos datos de entrenamiento y validación

Creamos dos particiones una de entrenamiento y otra de validación. Donde esta última, son los datos después de cierta fecha.

Modelo

Vemos el resultado sobre el set de datos

In [ ]:
lstm.eval()
train_predict = lstm(x)

data_predict = train_predict.cpu().data.numpy()
dataY_plot = y.cpu().data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)
#Dibujamos una linea roja que nos dira cuando comienza el set de validacion
plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.suptitle('Prediccion de la serie de tiempo')
plt.show()

# Laboratorio: Base de datos Shampoo

In [ ]:
training_set = pd.read_csv('shampoo.csv')
training_set.head()

In [ ]:
training_set = training_set.iloc[:,1:2].values
plt.plot(training_set )
plt.suptitle('Ventas de shampoo')
plt.show()

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)
seq_length = 4
x, y = sliding_windows(training_data, seq_length)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_size = int(len(y) * 0.67)
val_size = len(y) - train_size

x_train = torch.tensor(x[0:train_size],device=device)
y_train = torch.tensor(y[0:train_size],device=device)
x_val = torch.tensor(x[val_size:],device=device)
y_val = torch.tensor(y[val_size:],device=device)

x = torch.tensor(x,device=device)
y = torch.tensor(y,device=device)


In [ ]:
num_epochs = 2000
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

#por defecto se crea una lstm con precision de 16bit, en pytorch 1.6.0 hay un bug que no permite el uso de 16bit con lstm
#pasamos la red a doble precision
lstm = LSTM(num_classes, input_size, hidden_size, num_layers).to(device).double()

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(x_train)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, y_train)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
lstm.eval()
train_predict = lstm(x)

data_predict = train_predict.cpu().data.numpy()
dataY_plot = y.cpu().data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.suptitle('Time-Series Prediction')
plt.show()